## Adaptation de namedGame à Jupyter

Vous pouvez utiliser cette feuille pour visualiser un match.

Pour charger vos joueurs, il faut changer leurs noms dans 'classNames' (ligne 13), comme dans 'namedGame.py'.

Vous pouvez décommenter les lignes utilisant le librairie cairosvg pour générer un gif animé de votre match si vous le souhaitez (puis utilisez la ligne de commande donnée en commentaire pour transformer les PNG en GIF).

In [ ]:
 import  Goban 
 import  GnuGo
 import myPlayer 
 import myNeuralNetwork
 import myPlayer2 
 import playerInterface 
 import randomPlayer
 import time
 from io import StringIO
 import  sys
 from IPython.display  import  display, HTML, clear_output

In [ ]:
b = Goban.Board()

players = []

#player 1
player1 = myPlayer2.myPlayer()
player1.newGame(Goban.Board._BLACK)
players.append(player1)

#player 2
player2 = randomPlayer.myPlayer()
player2.newGame(Goban.Board._BLACK)
players.append(player2)

totalTime = [0,0] # total real time for each player
nextplayer = 0
nextplayercolor = Goban.Board._BLACK
nbmoves = 1

outputs = ["",""]
sysstdout= sys.stdout
stringio = StringIO()
wrongmovefrom = 0

print("The game is setup with " + player1.getPlayerName() + " playing BLACKS against " + player2.getPlayerName() + " playing WHITE")
print("Engines loaded. Ready to go.")


The game is setup with Carlos dos Santos playing BLACKS against Random Player playing WHITE
Engines loaded. Ready to go.


In [ ]:
coup = 1
while not b.is_game_over():
    print("Referee Board:")
    clear_output(wait=True)
    display(HTML(b.svg()))
    #Uncomment this for saving all the PNG images
    #svg2png(bytestring=b.svg(),write_to='output-'+str(coup).zfill(4)+'.png')
    legals = b.legal_moves() # legal moves are given as internal (flat) coordinates, not A1, A2, ...

    print("Legal Moves: ", [b.move_to_str(m) for m in legals]) # I have to use this wrapper if I want to print them
    nbmoves += 1
    otherplayer = (nextplayer + 1) % 2
    othercolor = Goban.Board.flip(nextplayercolor)
    
    currentTime = time.time()
    sys.stdout = stringio
    move = players[nextplayer].getPlayerMove() # The move must be given by "A1", ... "J8" string coordinates (not as an internal move)
    sys.stdout = sysstdout
    playeroutput = stringio.getvalue()
    stringio.truncate(0)
    stringio.seek(0)
    print(("[Player "+str(nextplayer) + "] ").join(playeroutput.splitlines(True)))
    outputs[nextplayer] += playeroutput
    totalTime[nextplayer] += time.time() - currentTime
    print("Player ", nextplayercolor, players[nextplayer].getPlayerName(), "plays: " + move) #changed 

    if not Goban.Board.name_to_flat(move) in legals:
        print(otherplayer, nextplayer, nextplayercolor)
        print("Problem: illegal move")
        wrongmovefrom = nextplayercolor
        break
    b.push(Goban.Board.name_to_flat(move)) # Here I have to internally flatten the move to be able to check it.
    players[otherplayer].playOpponentMove(move)
 
    nextplayer = otherplayer
    nextplayercolor = othercolor
    time.sleep(0.1)
    coup += 1

print("The game is over")
b.prettyPrint()
result = b.result()
print("Time:", totalTime)
print("GO Score:", b.final_go_score())
print("Winner: ", end="")
if wrongmovefrom > 0:
    if wrongmovefrom == b._WHITE:
        print("BLACK")
    elif wrongmovefrom == b._BLACK:
        print("WHITE")
    else:
        print("ERROR")
elif result == "1-0":
    print("WHITE")
elif result == "0-1":
    print("BLACK")
else:
    print("DEUCE")

# To convert png files to animated gif    
# convert -delay 10 -loop 0 *.png -background white -alpha remove -alpha off output.gif



Legal Moves:  ['D2', 'F2', 'D4', 'E5', 'J6', 'B9', 'G3', 'A5', 'B6', 'PASS']


AssertionError: ignored